In [1]:
import json
import hashlib
from hashlib import sha256
import time

In [2]:
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [3]:
class Blockchain: 
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.difficulty = 4
        self.create_genesis_block()
 
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    @property
    def last_block(self):
        return self.chain[-1]

    def proof_of_work(self, block):
        block.nonce = 1
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash    

    def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
 
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * self.difficulty) and
                block_hash == block.compute_hash())

    def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)
 
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
 
        last_block = self.last_block
 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index

In [4]:
tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}

In [5]:
tx_2 = {
   "addr_from": "1J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "2BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 199.00
}

In [6]:
Titans = Blockchain()
while len(Titans.chain)<10:
  Titans.add_new_transaction([tx_1, tx_2])
  Titans.mine()
  print(Titans.chain[-1].hash)
  print(str(Titans.chain[-1].nonce))


00001ead712395213810f99f094c208635c0337ffb91d6f6d252644457ea8cba
3977
000038d22c58915188f6637c49ba6b6e63842e5560aeab6210cbdf926dc6ffb1
99951
0000ccbdd1bbfd2565efb5110ee716719f9fb152d1053c3ce921cd0baddfbd90
53344
00008ca3adfa1762de78a7e8871780122354300345ca2d5d09827d1e6179893f
55324
000058a1a1684778f33620c34f95f1dc9e326ab20f71cfdf2a4632568c66a6cd
83085
00006b33af5126510852e87ca894a85837944d4ae8f0aaf5210082d2dbcd4048
33276
0000aa97ca4abc98b1060d533a9349104810df4f39debdfdacc5a7affa720672
163652
0000c04a89556a9c9061029945c7eeeb3890aa17591a791e85d63ec6656efa58
199166
0000d07af91fdec917320e1568394d89aca90bd4e9ddd00bd0dc87e632477566
95123
